In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [8]:
input_df = pd.read_csv("Input.csv", header=None)

In [9]:
input_df

,0
0,0.707403
1,0.710485
2,0.720442
3,0.727968
4,0.740602
...,...
379,0.212020
380,0.213939
381,0.219018
382,0.224313


In [3]:
# Carregando CNN
cnn_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
cnn_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 384, 1)]          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 384, 32)           224       
                                                                 
 spatial_dropout1d_3 (Spatia  (None, 384, 32)          0         
 lDropout1D)                                                     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 128, 32)          0         
 1D)                                                             
                                                                 
 batch_normalization_3 (Batc  (None, 128, 32)          128       
 hNormalization)                                                 
                                                           

In [12]:
# Carregando Dataset
datasetCon = np.loadtxt("data.csv", delimiter=",")

"""60% - Treino, 20% - Validação, 20% - Teste"""

# Coleta os dados das entradas datasetCon (ficando de fora os dados do One-hot-encoder)
x_datasetCon = datasetCon[:, :-1]

# Coleta os dados das saídas
y_datasetCon = datasetCon[:, -1]

# Separa os dados de entrada e saida para treino e testes
x_train,x_temp,y_train,y_temp = train_test_split(x_datasetCon,y_datasetCon, test_size=0.3, random_state=42, shuffle=True, stratify=y_datasetCon)
x_test,x_validation,y_test,y_validation = train_test_split(x_temp,y_temp, test_size=0.5, random_state=42, shuffle=True, stratify=y_temp)



"""Tamanho do teste e Validação (X, Y)"""

# Treino
print(x_train.shape, y_train.shape)

# Validação
print(x_validation.shape, y_validation.shape)

# Test
print(x_test.shape, y_test.shape)

"""Normalização"""

scaler  = MinMaxScaler(feature_range=(0,1)) # Tangente Hiperbólica ou outro análoga

x_train       = scaler.fit_transform(x_train)
x_validation  = scaler.fit_transform(x_validation)
x_test        = scaler.fit_transform(x_test)

(27104, 384) (27104,)
(5808, 384) (5808,)
(5808, 384) (5808,)


In [13]:
# Evaluate the restored model
loss, acc = cnn_model.evaluate(x_test, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(cnn_model.predict(x_test).shape)

182/182 - 1s - loss: 0.5617 - acc: 0.8289 - 1s/epoch - 8ms/step
Restored model, accuracy: 82.89%
182/182 [==============================] - 1s 5ms/step
(5808, 40)


In [15]:
# **Validação da base de teste**
saida_rede = pd.DataFrame(cnn_model.predict(x_test)).round(decimals = 2)

display(saida_rede)

182/182 [==============================] - 1s 5ms/step


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.00,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,...,0.97,0.01,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00
1,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.99,0.0,0.0,0.0,0.00,0.00
2,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.00,0.41,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.32
3,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00
4,0.66,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.00,0.00,0.03,0.0,0.00,0.0,0.0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5803,0.99,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00
5804,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.5,0.00,0.00
5805,0.00,0.0,0.0,0.0,0.0,0.0,0.16,0.0,0.0,0.0,...,0.77,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.02,0.00
5806,0.01,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00


In [17]:
# **Acerto da rede**
predito   = saida_rede.idxmax(axis=1)
predito

0       30
1       34
2       31
3       16
4        0
        ..
5803     0
5804    37
5805    30
5806    24
5807    38
Length: 5808, dtype: int64

In [45]:
datasetCon_scaled = scaler.fit_transform(x_datasetCon)

x = datasetCon_scaled[0,:384].reshape(1,384)

In [53]:
(pd.DataFrame(cnn_model.predict(x))).round(decimals = 3)

1/1 [==============================] - 0s 21ms/step


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.978,0.0,0.0,0.0,0.0,0.0,0.0,0.002,0.004,0.0,...,0.0,0.0,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
cnn_model.predict(x)

1/1 [==============================] - 0s 43ms/step


array([[9.77775395e-01, 3.75174859e-05, 9.02735155e-06, 1.55253218e-07,
        3.23324889e-06, 9.04231347e-05, 5.19225687e-05, 2.09588977e-03,
        3.96302016e-03, 3.14395729e-05, 2.59405670e-08, 3.84435737e-07,
        2.37972540e-06, 3.23836684e-05, 2.53927701e-05, 6.64742911e-05,
        2.54042866e-03, 1.74341214e-04, 3.09301208e-06, 2.05391621e-06,
        4.08006526e-05, 5.76450475e-06, 1.91700456e-05, 4.17932548e-04,
        1.07391095e-02, 3.81842219e-05, 1.30073772e-06, 7.57271401e-09,
        1.74377710e-05, 2.95174577e-05, 1.06005655e-05, 2.73756777e-05,
        1.17994030e-03, 2.00228205e-05, 1.89297918e-06, 2.71042472e-06,
        7.84553413e-05, 4.01469333e-06, 1.46181528e-05, 4.46112186e-04]],
      dtype=float32)

In [56]:
datasetCon_scaled[0, :]

(384,)

In [63]:
np.savetxt("Input_2.csv", datasetCon_scaled[0:5, :], delimiter=",")

In [4]:
np.loadtxt("Input.csv", delimiter=",").shape[0]

384